In [1]:
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle fastai py7zr scikit-learn

In [2]:
from fastkaggle import *
import gc

In [3]:
# for working with paths in Python, I recommend using `pathlib.Path`
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not iskaggle and not cred_path.exists():
    creds = ''
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [4]:
comp = 'kkbox-music-recommendation-challenge'

path = setup_comp(comp, install='fastai')

100%|██████████| 345M/345M [00:06<00:00, 56.4MB/s] 


In [5]:
path

Path('kkbox-music-recommendation-challenge')

In [6]:
from fastai.tabular.all import *
from fastai.collab import *

path.ls()

(#6) [Path('kkbox-music-recommendation-challenge/train.csv.7z'),Path('kkbox-music-recommendation-challenge/members.csv.7z'),Path('kkbox-music-recommendation-challenge/songs.csv.7z'),Path('kkbox-music-recommendation-challenge/test.csv.7z'),Path('kkbox-music-recommendation-challenge/song_extra_info.csv.7z'),Path('kkbox-music-recommendation-challenge/sample_submission.csv.7z')]

In [7]:
def is_extracted(file):
    return os.path.isfile(os.path.splitext(file)[0])

for file in path.ls(file_exts=".7z"):
    if not is_extracted(file):
        !py7zr x {str(file)} {str(path)}

In [8]:
path.ls(file_exts=".csv")

(#6) [Path('kkbox-music-recommendation-challenge/songs.csv'),Path('kkbox-music-recommendation-challenge/song_extra_info.csv'),Path('kkbox-music-recommendation-challenge/sample_submission.csv'),Path('kkbox-music-recommendation-challenge/members.csv'),Path('kkbox-music-recommendation-challenge/test.csv'),Path('kkbox-music-recommendation-challenge/train.csv')]

In [9]:
songs = pd.read_csv(path/'train.csv', usecols=('msno', 'song_id', 'target'))

songs.head()

,msno,song_id,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,1


In [10]:
dls = CollabDataLoaders.from_df(songs, bs=512)
dls.show_batch()

,msno,song_id,target
0,FWlfL4RQkWXDTzs5+/l5hQKEJTT2owvu6EifcFdtggA=,At7ayZb8zrkdvXw+EBt7z5e4IZRe0JhfOC8HqzzA0sg=,1
1,Jd8Z/drETPjM3LGQ9HXyny+5G4VY76/2eEKAHQEGGWI=,E/b+7QWv/HguM/u4uQXY1/2jeiFsxst6FRK2shsNHiU=,0
2,I6BWuX+5GHb/f6O+y9Bt97A9KrX4hzn6Tj4Ga8jz/xM=,ZISVKxx7B/9L73xZ4xlQluOixjbqBMxyG3q+kEi76oY=,1
3,jft7pSzm46OHjKJxMeXWSdtORRv1eBxThVAhJq+icwY=,JTMsJNPcL2ambkp1Z/MeBEnFEzZEOKetdLPrJxHf7v8=,0
4,GL6IfpVeklljPUNgIwOh4j7MYDNwK7H+bg/w+W+oO5E=,GKgBgCnz9R8OWDdGeYgQS0TPHe8no8VWvxj4+4PSjao=,1
5,OtFfoISRCGiLHdMnyWFSMzN9SlwT7y8G4l92sMWYN6g=,+Sm75wnBf/sjm/QMUAFx8N+Ae04kWCXGlgH50tTeM6c=,0
6,2aY4fVvcDt6tfVFp2DJZ5gEXWlMPTUQWUgEe9Fut0mw=,CYQ38DzaKqRnuEfAB5kuQmFborYHWzbqO/7Bvny/2Mg=,0
7,/A6sqt60VJBGT59/JZXPvBXKY9uW/5GofKHv8f9hKco=,4weA5ndJkDSo7zM38+4t7NT+CJ/5omnZYVHf/IDe+Qk=,1
8,yCCT+QckrmYll48S7BBzCLGn8HekGPwXsTaRYpV5qCk=,IWEzXZ8nCiKZHbQl9fJmOjYRxHfSkhcm7AQbV5j4/J0=,0
9,5/kRuQO50xlgEWz7pNQLhlQnpE0sjua+gjrmy8X+ues=,RBu+exsgC2/E1g0e+i6UnQmRseAUB8cdznbQu2Tafco=,1


In [11]:
from sklearn.metrics import roc_auc_score

def roc_metric(pred, target):
    return roc_auc_score(to_np(target.squeeze()), to_np(torch.clip(pred,0,1)))

learn = collab_learner(dls, y_range=(-0.1, 1.1), metrics=roc_metric)

In [12]:
learn.fit_one_cycle(1, 1e-1, wd=0.1)

epoch,train_loss,valid_loss,roc_metric,time
0,0.211276,0.217323,0.720072,02:04


In [15]:
torch.save(learn.model.state_dict(), '01.pth')
# learn.model.load_state_dict(torch.load('01.pth'))
# learn.model.load_state_dict(torch.load('01.pth', map_location=torch.device('cpu')))

In [16]:
learn.model

EmbeddingDotBias(
  (u_weight): Embedding(30756, 50)
  (i_weight): Embedding(359967, 50)
  (u_bias): Embedding(30756, 1)
  (i_bias): Embedding(359967, 1)
)

In [17]:
def merge_data(songs):
    song_data_0 = pd.read_csv(path/'songs.csv')
    with_song_0 = pd.merge(songs, song_data_0, how='left', on='song_id')
    del song_data_0
    del songs
    gc.collect()
    
    song_data_1 = pd.read_csv(path/'song_extra_info.csv')
    with_song_1 = pd.merge(with_song_0, song_data_1, how='left', on='song_id')
    del with_song_0
    del song_data_1
    gc.collect()
    
    members = pd.read_csv(path/'members.csv')
    with_members = pd.merge(with_song_1, members, how='left', on='msno')
    del with_song_1
    del members
    gc.collect()
    
    song_bias_df = pd.DataFrame({
        'song_id': learn.dls.classes['song_id'],
        'song_bias': to_np(learn.model.i_bias.weight.squeeze()),
    })
    song_weights_df = pd.DataFrame([
        {
            'song_id': song_id,
            **{f'song_weight_{i+1}': weight for i, weight in enumerate(weights)}
        }
        for song_id, weights in zip(learn.dls.classes['song_id'], learn.model.i_weight.weight.tolist())
    ])
    song_embeddings_df = pd.merge(song_bias_df, song_weights_df, on='song_id', how='left')

    del song_bias_df
    del song_weights_df
    gc.collect()

    with_song_embeddings = pd.merge(with_members, song_embeddings_df, how='left', on='song_id')
    del with_members
    del song_embeddings_df
    gc.collect()
    
    user_bias_df = pd.DataFrame({
        'msno': learn.dls.classes['msno'],
        'user_bias': to_np(learn.model.u_bias.weight.squeeze()),
    })
    user_weights_df = pd.DataFrame([
        {
            'msno': msno,
            **{f'user_weight_{i+1}': weight for i, weight in enumerate(weights)}
        }
        for msno, weights in zip(learn.dls.classes['msno'], learn.model.u_weight.weight.tolist())
    ])
    user_embeddings_df = pd.merge(user_bias_df, user_weights_df, on='msno', how='left')

    del user_bias_df
    del user_weights_df
    gc.collect()

    
    with_member_embeddings = pd.merge(with_song_embeddings, user_embeddings_df, how='left', on='msno')
    del with_song_embeddings
    del user_embeddings_df
    gc.collect()
    
    all_data = with_member_embeddings
    
    del all_data['expiration_date']
    del all_data['registered_via']
    del all_data['isrc']
    all_data = add_datepart(all_data, 'registration_init_time')
    gc.collect()

    return all_data

all_data = merge_data(songs)

gc.collect()

all_data.head()

,msno,song_id,target,song_length,genre_ids,artist_name,composer,lyricist,language,name,...,registration_init_timeDay,registration_init_timeDayofweek,registration_init_timeDayofyear,registration_init_timeIs_month_end,registration_init_timeIs_month_start,registration_init_timeIs_quarter_end,registration_init_timeIs_quarter_start,registration_init_timeIs_year_end,registration_init_timeIs_year_start,registration_init_timeElapsed
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,1,206471.0,359,Bastille,Dan Smith| Mark Crew,NaN,52.0,Good Grief,...,1,3,1,False,True,False,True,False,True,0.0
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,1,284584.0,1259,Various Artists,NaN,NaN,52.0,Lords of Cardboard,...,1,3,1,False,True,False,True,False,True,0.0
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,1,225396.0,1259,Nas,N. Jones、W. Adams、J. Lordan、D. Ingle,NaN,52.0,Hip Hop Is Dead(Album Version (Edited)),...,1,3,1,False,True,False,True,False,True,0.0
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,1,255512.0,1019,Soundway,Kwadwo Donkoh,NaN,-1.0,Disco Africa,...,1,3,1,False,True,False,True,False,True,0.0
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,1,187802.0,1011,Brett Young,Brett Young| Kelly Archer| Justin Ebach,NaN,52.0,Sleep Without You,...,1,3,1,False,True,False,True,False,True,0.0


In [20]:
cat_names = ['msno', 'song_id', 'genre_ids', 'artist_name', 'composer', 'lyricist', 'language', 'name', 'city', 'gender', 'registration_init_timeYear', 'registration_init_timeMonth', 'registration_init_timeWeek', 'registration_init_timeDay', 'registration_init_timeDayofweek', 'registration_init_timeDayofyear', 'registration_init_timeIs_month_end', 'registration_init_timeIs_month_start', 'registration_init_timeIs_quarter_end', 'registration_init_timeIs_quarter_start', 'registration_init_timeIs_year_end', 'registration_init_timeIs_year_start']
cont_names = ['bd', 'song_length', 'registration_init_timeElapsed', 'song_bias', 'song_weight_1', 'song_weight_2', 'song_weight_3', 'song_weight_4', 'song_weight_5', 'song_weight_6', 'song_weight_7', 'song_weight_8', 'song_weight_9', 'song_weight_10', 'song_weight_11', 'song_weight_12', 'song_weight_13', 'song_weight_14', 'song_weight_15', 'song_weight_16', 'song_weight_17', 'song_weight_18', 'song_weight_19', 'song_weight_20', 'song_weight_21', 'song_weight_22', 'song_weight_23', 'song_weight_24', 'song_weight_25', 'song_weight_26', 'song_weight_27', 'song_weight_28', 'song_weight_29', 'song_weight_30', 'song_weight_31', 'song_weight_32', 'song_weight_33', 'song_weight_34', 'song_weight_35', 'song_weight_36', 'song_weight_37', 'song_weight_38', 'song_weight_39', 'song_weight_40', 'song_weight_41', 'song_weight_42', 'song_weight_43', 'song_weight_44', 'song_weight_45', 'song_weight_46', 'song_weight_47', 'song_weight_48', 'song_weight_49', 'song_weight_50', 'user_bias', 'user_weight_1', 'user_weight_2', 'user_weight_3', 'user_weight_4', 'user_weight_5', 'user_weight_6', 'user_weight_7', 'user_weight_8', 'user_weight_9', 'user_weight_10', 'user_weight_11', 'user_weight_12', 'user_weight_13', 'user_weight_14', 'user_weight_15', 'user_weight_16', 'user_weight_17', 'user_weight_18', 'user_weight_19', 'user_weight_20', 'user_weight_21', 'user_weight_22', 'user_weight_23', 'user_weight_24', 'user_weight_25', 'user_weight_26', 'user_weight_27', 'user_weight_28', 'user_weight_29', 'user_weight_30', 'user_weight_31', 'user_weight_32', 'user_weight_33', 'user_weight_34', 'user_weight_35', 'user_weight_36', 'user_weight_37', 'user_weight_38', 'user_weight_39', 'user_weight_40', 'user_weight_41', 'user_weight_42', 'user_weight_43', 'user_weight_44', 'user_weight_45', 'user_weight_46', 'user_weight_47', 'user_weight_48', 'user_weight_49', 'user_weight_50']
dep = 'target'

In [ ]:
def tabularify(data, is_train=True):
    y_names = None;
    splits = None;
    
    if is_train:
        y_names = dep;
        splits = RandomSplitter(valid_pct=0.2)(range_of(data));
        
    return TabularPandas(data,
                         procs=[Categorify, FillMissing,Normalize],
                         cat_names=cat_names,
                         cont_names=cont_names,
                         y_names=y_names,
                         splits=splits)

to = tabularify(all_data)

X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_valid, y_valid = to.valid.xs, to.valid.ys.values.ravel()

del all_data
del to

gc.collect()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef

rf = RandomForestClassifier(100, min_samples_leaf=5)
rf.fit(X_train, y_train)

del X_train
del y_train
gc.collect()

matthews_corrcoef(y_valid, rf.predict(X_valid))

In [ ]:
del X_valid
del y_valid
gc.collect()

test_df = pd.read_csv(path/'test.csv', usecols=('msno', 'song_id', 'id'))

test_data = merge_data(test_df)

test_data.head()

In [ ]:
to_test = tabularify(test_data, is_train=False)
X_test = to_test.train.xs

In [ ]:
output = rf.predict(X_test)

del to_test
del X_test
gc.collect()

test_df['target'] = torch.clip(output, 0, 1)

In [ ]:
sample_submission = pd.read_csv(path/'sample_submission.csv')
sample_submission

In [ ]:
test_df['target'] = torch.clip(output, 0, 1)

sub_df = test_df[['id','target']]
sub_df.to_csv(f'submission.csv', index=False)

In [ ]:
!head submission.csv

In [ ]:
if not iskaggle:
    from kaggle import api
    api.competition_submit_cli('submission.csv', 'kkbox-recommendation-6', comp)

In [5]:
if not iskaggle:
    push_notebook('alexchalk', 'poisonous-mushrooms-submission-05',
                  title='Poisonous Mushrooms Submission 05',
                  file='02.ipynb',
                  competition=comp, private=True, gpu=False)

Kernel version 1 successfully pushed.  Please check progress at https://www.kaggle.com/code/alexchalk/poisonous-mushrooms-submission-05
